Run the requirements.txt file to download dependencies needed

# Run this cell if you want to use Ollama engine:

In [2]:
# RUN THIS CELL IF YOU WANT TO USE OLLAMA ENGINE
from llama_index.llms.ollama import Ollama
from llama_index.core.indices.struct_store import JSONQueryEngine

# llm = Ollama(model="llama3", request_timeout=120.0)

# Test setup
# resp = llm.complete("What is Mcdonalds?")
# print(resp)

# Begin vector embedding of scrapped data

In [1]:
import json

# Function to read JSON data from a file
def read_json_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return json.load(file)

In [2]:
# Paths to the JSON files
list_of_json_files = [
    "cp_data/tenant_desc.json",
    "cp_data/tenant_schema.json",
    "cp_data/about_us.json",
    "cp_data/news.json",
    "cp_data/facilities_desc.json",
    "cp_data/whats_hot.json",
]

In [3]:
from llama_index.readers.json import JSONReader

reader = JSONReader()

# Load data from multiple files
documents = []
for file_path in list_of_json_files:
    documents.extend(reader.load_data(file_path))

for doc in documents:
    print("Document ID:", doc.doc_id)

Document ID: b541f911-772d-4acf-828f-d728f08e890a
Document ID: 6b45cf56-4eee-4197-8c5c-f70fd5bc7b2c
Document ID: f59a8a5d-eff1-4582-9fac-d38e7ba820a3
Document ID: 07a3c015-05f6-47de-8e16-329e36d3e1da
Document ID: cd37c002-ee02-4b60-b966-55db04d20fc8
Document ID: 09c433cd-bc4e-4bb5-8a78-d4aab71e0659


In [4]:
# Create an index over the documents
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
import os
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

api_key = os.getenv("OPENAI_API_KEY")

# Setup MilvusVectorStore
vector_store = MilvusVectorStore(
    dim=1536,
    collection_name="cp_tenant_vector",
    uri="./milvus_demo.db",
    token="root:Milvus",
    overwrite=True
)

# Create a storage context with the Milvus vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index from your documents
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

/opt/miniconda3/envs/v2_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 58/58 [00:04<00:00, 13.18it/s]


# Uncomment this cell to test project setup 

In [5]:
# Uncomment to Run Test:

# query_engine = index.as_query_engine(llm=llm) #uncomment to use ollama llm
query_engine = index.as_query_engine()

res1 = query_engine.query("What is CHATERAISE category and where is its location")
res2 = query_engine.query("Who is the US landscape consultant for central park mall")
print(res1)
print(res2)

CHATERAISE is in the "Gourmet Delights" category and its location is on LG 119 – LG 120.
Bennit Design Group - USA


# Run this cell for FAST API deployment and running RAG with RESTFUL env:

This call only if you choose to run Fast API with ngrok or you can deploy locally

!ngrok authtoken "YOUR API KEY HERE"

In [6]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import threading 
import uvicorn
import logging

class QueryResponse(BaseModel):
    response: str

class QueryRequest(BaseModel):
    query: str

# Initialize the FastAPI app
app = FastAPI()

llama_index = query_engine

@app.post("/query", response_model=QueryResponse)
async def query_llama_index(request: QueryRequest):
    try:
        # Logging the received request
        logging.info(f"Received query: {request.query}")

        # Extract the 'query' value from the request body
        query = request.query
        result = llama_index.query(query)
        if isinstance(result, str):
            return QueryResponse(response=result)
        else:
            return QueryResponse(response=result.response)
    except Exception as e:
        logging.error(f"Error occurred: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {str(e)}")


# Function to run Uvicorn server
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the server in a background thread
server = threading.Thread(target=run)
server.start()



INFO:     Started server process [67973]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:53546 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53546 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53549 - "POST /query HTTP/1.1" 200 OK
INFO:     127.0.0.1:53783 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53783 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53783 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53783 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:53783 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53783 - "GET /openapi.json HTTP/1.1" 200 OK
